## Custom Dijkstra
- find all combinations between two points given minimum number of stops

In [1]:
# Dependencies  
from collections import deque
from heapq import heappop, heappush
from itertools import count

import networkx as nx
from networkx.algorithms.shortest_paths.generic import _build_paths_from_predecessors


In [2]:
def _weight_function(G, weight):
    if callable(weight):
        return weight
    # If the weight keyword argument is not callable, we assume it is a
    # string representing the edge attribute containing the weight of
    # the edge.
    if G.is_multigraph():
        return lambda u, v, d: min(attr.get(weight, 1) for attr in d.values())
    return lambda u, v, data: data.get(weight, 1)



In [3]:
def multi_source_dijkstra(G, sources, target=None, cutoff=None, weight="weight"):
    sources = {sources}
    if not sources:
        raise ValueError("sources must not be empty")
    for s in sources:
        if s not in G:
            raise nx.NodeNotFound(f"Node {s} not found in graph")
    if target in sources:
        return (0, [target])
    weight = _weight_function(G, weight)
    
    paths = {source: [source] for source in sources}  # dictionary of paths
 
    total_distance = _dijkstra_multisource(
        G, sources, weight, paths=paths, cutoff=cutoff, target=target
    )
   
    if target is None:
        return (total_distance, paths)
    try:
        return (total_distance[target], paths[target])
    except KeyError as err:
        raise nx.NetworkXNoPath(f"No path to {target}.") from err

In [4]:
def _dijkstra(G, source, weight, pred=None, paths=None, cutoff=None, target=None):
    return _dijkstra_multisource(
        G, [source], weight, pred=pred, paths=paths, cutoff=cutoff, target=target
    )

In [5]:
'''
NOTES:  
Takes source and stores it in visited and adds to queue (initialization)
While loop
  pop queue
  take source (v); make sure node not traversed
  get destinations (u) via loop
  add destination 'cost/weight' to temporary distance variable
  update destination node (u) distance 
  update visited with u
  add to queue the next node
  update destination list
'''

"\nNOTES:  \nTakes source and stores it in visited and adds to queue (initialization)\nWhile loop\n  pop queue\n  take source (v); make sure node not traversed\n  get destinations (u) via loop\n  add destination 'cost/weight' to temporary distance variable\n  update destination node (u) distance \n  update visited with u\n  add to queue the next node\n  update destination list\n"

In [6]:
# Customized dijkstra from:  
## starting code:  https://networkx.org/documentation/stable/_modules/networkx/algorithms/shortest_paths/weighted.html#multi_source_dijkstra 

def _dijkstra_multisource(
    G, sources, weight, pred=None, paths=None, cutoff=None, target=None
):
    # G: graph of nodes
    # sources: list of starting node(s)
    # weight: weight function - minimized value
    # pred: predecessors - dict of lists
    # paths: dict holds list of nodes making path
    # cutoff: maximum weight before search ends
    # target: ending node
    
    G_succ = G._adj  # For speed-up (and works for both directed and undirected graphs)
    push = heappush
    pop = heappop
    c = count()  # iterator
    
    # tracking progress
    total_distance = {}  # dictionary of final distance
    visited = {}  # nodes that have been visited
    queue = []  # queue is heapq with 3-tuples (distance,c,node)
    
    # intilize heap with source(s)
    for initial_location in sources:
        visited[initial_location] = 0
        push(queue, (0, next(c), initial_location))
        
    # search each node
    while queue:
        # select source - given distance and source
        (path_distance, _, source) = pop(queue)
        
        # already searched this node.
        if source in total_distance:
            continue  
            
        # add distance associated with node (v)
        total_distance[source] = path_distance  
        
        # found destination
        if source == target:
            break
            
        # From node(v) loop through child 
        # destination: destination node, e: edge data, v: source node
        for destination, edge_data in G_succ[source].items():
            
            # very important - tells how minimization occurs
            cost = weight(source, destination, edge_data)
            print(source, destination, edge_data, 'cost: ', cost)
            
            # no other paths
            if cost is None:
                continue
                
            # sum past distance with new cost and assign to this loop variable
            source_to_destination_distance = total_distance[source] + cost  
                    
            #  check if negative weights
            if destination in total_distance:
                # grab existing node distance
                destination_distance = total_distance[destination]
                
                # check if source-destination distance is less than the 
                if source_to_destination_distance < destination_distance:
                    raise ValueError("Contradictory paths found:", "negative weights?")  
                    
            # not in visited(aka source initally) or vu distance less than destination distance
            elif destination not in visited or source_to_destination_distance < visited[destination]:
                
                # update visited source node distance
                visited[destination] = source_to_destination_distance
                
                # update queue to include new distance, iterator, and destination
                # basically recursion but done by while loop checking queue
                push(queue, (source_to_destination_distance, next(c), destination))
                
                # paths is passed source node initially
                # the paths referenced below is a global variable
                if paths is not None:
                    paths[destination] = paths[source] + [destination]
                    
    return total_distance

In [7]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from scipy.spatial.distance import cdist

# Custom graph data
graph = {
    'London': {'Paris': {'population': 2141000, 'lat': 48.8566, 'lon': 2.3522}, 'New_York': {'population': 8399000, 'lat': 40.7128, 'lon': -74.0060}, 'Rio_de_Janeiro': {'population': 6748000, 'lat': -22.9068, 'lon': -43.1729}},
    'Paris': {'Dubai': {'population': 3330000, 'lat': 25.2048, 'lon': 55.2708}, 'Rio_de_Janeiro': {'population': 6748000, 'lat': -22.9068, 'lon': -43.1729}, 'London': {'population': 8982000, 'lat': 51.5074, 'lon': -0.1278}, 'New_York': {'population': 8399000, 'lat': 40.7128, 'lon': -74.0060}, 'Rio_de_Janeiro': {'population': 6748000, 'lat': -22.9068, 'lon': -43.1729}},
    'New_York': {'Los_Angeles': {'population': 3990000, 'lat': 34.0522, 'lon': -118.2437}},
    'Dubai': {'Rio_de_Janeiro': {'population': 6748000, 'lat': -22.9068, 'lon': -43.1729}, 'Tokyo': {'population': 13960000, 'lat': 35.6895, 'lon': 139.6917}},
    'Rio_de_Janeiro': {'New_York': {'population': 8399000, 'lat': 40.7128, 'lon': -74.0060}},
    'Tokyo': {'Dubai': {'population': 3330000, 'lat': 25.2048, 'lon': 55.2708}, 'Los_Angeles': {'population': 3990000, 'lat': 34.0522, 'lon': -118.2437}},
    'Los_Angeles': {'New_York': {'population': 8399000, 'lat': 40.7128, 'lon': -74.0060}, 'Tokyo': {'population': 13960000, 'lat': 35.6895, 'lon': 139.6917}}
}

# Create network graph
G = nx.Graph()

# create dictionary of city and locations
# Calculate pairwise distances between cities using Haversine formula
city_lookup={}
for source, data in graph.items():
    for destination, info in data.items():
        city_lookup[source] = [info['lat'], info['lon']]


# Add nodes and edges
for source, destination_data in graph.items():
    G.add_node(source)
    for destination, data in destination_data.items():
        G.add_node(destination)
        locations = np.stack((city_lookup[destination], city_lookup[source]))
        distances = cdist(np.radians(locations), np.radians(locations), metric='euclidean') * 6371  # Earth radius in km
        # create edges
        G.add_edge(source, destination, population=data['population'], distance=distances[0,1])


# Calculate population per distance for each edge
for u, v, data in G.edges(data=True):
    distance = data['distance']
    population = data['population']
    data['pop_per_distance'] = population / distance

# Set the minimum number of cities in the path
min_num_cities = 1
start_city = 'London'
end_city = 'Los_Angeles'

# Find the path with the largest population per distance traveled while ensuring minimum cities
shortest_paths = multi_source_dijkstra(G, start_city, end_city, weight='pop_per_distance')
print('final: ', shortest_paths)

London Paris {'population': 8982000, 'distance': 7861.201009301914, 'pop_per_distance': 1142.5735061820553} cost:  1142.5735061820553
London New_York {'population': 8399000, 'distance': 10478.287221079576, 'pop_per_distance': 801.562299523858} cost:  801.562299523858
London Rio_de_Janeiro {'population': 6748000, 'distance': 7861.201009301914, 'pop_per_distance': 858.3930104338131} cost:  858.3930104338131
New_York London {'population': 8399000, 'distance': 10478.287221079576, 'pop_per_distance': 801.562299523858} cost:  801.562299523858
New_York Paris {'population': 8399000, 'distance': 4974.451033454915, 'pop_per_distance': 1688.427515622086} cost:  1688.427515622086
New_York Los_Angeles {'population': 8399000, 'distance': 28681.685706832082, 'pop_per_distance': 292.83495000432725} cost:  292.83495000432725
New_York Rio_de_Janeiro {'population': 8399000, 'distance': 4974.451033454915, 'pop_per_distance': 1688.427515622086} cost:  1688.427515622086
Rio_de_Janeiro London {'population': 

C:\Users\ejwda\AppData\Local\Temp\ipykernel_16256\2030934238.py:43: RuntimeWarning: divide by zero encountered in double_scalars
  data['pop_per_distance'] = population / distance
